<a href="https://colab.research.google.com/github/vignesh312000/Deep_Learning-_CNN/blob/main/Rediminds_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python

In [ ]:
import cv2
import os
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define parameters
num_samples_per_class = 200  # Number of videos per class
num_frames = 10  # Number of frames per video
image_width, image_height = 224, 224  # Image dimensions
num_classes = 2  # Number of classes (violence and non-violence)
BATCH_SIZE = 8  # Reduce batch size


In [ ]:
def load_video_data(folder_path, label, num_samples):
    video_data = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.mp4'):
            cap = cv2.VideoCapture(os.path.join(folder_path, filename))
            frames = []
            while len(frames) < num_samples:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (image_width, image_height))
                frame = frame.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
                frames.append(frame)
            cap.release()
            if len(frames) == num_samples:
                video_data.append(frames)
                labels.append(label)
    return np.array(video_data), np.array(labels)


In [ ]:
def preprocess_video(video_file, target_width, target_height):
    frames = []
    cap = cv2.VideoCapture(video_file)
    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (target_width, target_height))
        frame = frame.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
        frames.append(frame)
    cap.release()
    return np.array(frames)


In [ ]:
non_violence_folder = '/content/drive/MyDrive/N_V'
violence_folder = '/content/drive/MyDrive/V'

In [ ]:
# Load video data for non-violence videos
non_violence_data, non_violence_labels = load_video_data(non_violence_folder, label=0, num_samples=num_frames)

# Load video data for violence videos
violence_data, violence_labels = load_video_data(violence_folder, label=1, num_samples=num_frames)



In [ ]:
# Concatenate the data from both classes
X = np.concatenate([non_violence_data, violence_data], axis=0)
y = np.concatenate([non_violence_labels, violence_labels])

In [ ]:
print("Number of samples in dataset:", len(X))

Number of samples in dataset: 400


In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_onehot = to_categorical(y, num_classes)

In [ ]:
# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

In [ ]:
# Reshape the data for the model
X_train_reshaped = X_train.reshape(-1, num_frames, image_height, image_width, 3)
X_val_reshaped = X_val.reshape(-1, num_frames, image_height, image_width, 3)

In [ ]:
print("X_train_reshaped shape:", X_train_reshaped.shape)
print("y_train shape:", y_train.shape)
print("X_train_reshaped dtype:", X_train_reshaped.dtype)
print("y_train dtype:", y_train.dtype)

X_train_reshaped shape: (320, 10, 224, 224, 3)
y_train shape: (320, 2)
X_train_reshaped dtype: float32
y_train dtype: float32


In [ ]:
# Define function to generate batches of data
def data_generator(X, y, batch_size):
    num_samples = len(X)
    num_batches = num_samples // batch_size
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        yield X[start_idx:end_idx], y[start_idx:end_idx]

In [ ]:
# Define the TensorFlow Dataset for training
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(X_train_reshaped, y_train, BATCH_SIZE),
    output_signature=(
        tf.TensorSpec(shape=(None, num_frames, image_height, image_width, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
).cache().shuffle(buffer_size=10000).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Print shapes of input and output tensors of the dataset
for x_batch, y_batch in train_dataset.take(1):
    print("Shape of input batch:", x_batch.shape)
    print("Shape of output batch:", y_batch.shape)

Shape of input batch: (8, 10, 224, 224, 3)
Shape of output batch: (8, 2)


In [ ]:
#  Define CNN model architecture
model = models.Sequential([
    # Convolutional layers
    layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(10, 224, 224, 3)),
    layers.MaxPooling3D((1, 2, 2)),  # Reduced downsampling
    layers.Conv3D(64, (3, 3, 3), activation='relu'),
    layers.MaxPooling3D((1, 2, 2)),
    layers.Conv3D(128, (3, 3, 3), activation='relu'),
    layers.MaxPooling3D((1, 2, 2)),
    layers.Conv3D(128, (3, 3, 3), activation='relu'),
    layers.MaxPooling3D((1, 2, 2)),
    layers.Flatten(),
    # Fully connected layers
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust num_classes based on your task
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
for x_batch, y_batch in train_dataset:
    print("Input shape:", x_batch.shape)
    print("Output shape:", y_batch.shape)
    break  # Only print the shape of the first batch

Input shape: (8, 10, 224, 224, 3)
Output shape: (8, 2)


In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=10)



Epoch 1/10
40/40 [==============================] - 916s 23s/step - loss: 0.7694 - accuracy: 0.5344
Epoch 2/10
40/40 [==============================] - 906s 23s/step - loss: 0.5713 - accuracy: 0.6844
Epoch 3/10
40/40 [==============================] - 907s 23s/step - loss: 0.5293 - accuracy: 0.6719
Epoch 4/10
40/40 [==============================] - 906s 23s/step - loss: 0.5045 - accuracy: 0.7437
Epoch 5/10
40/40 [==============================] - 903s 23s/step - loss: 0.4768 - accuracy: 0.7563
Epoch 6/10
40/40 [==============================] - 894s 22s/step - loss: 0.4484 - accuracy: 0.7937
Epoch 7/10
40/40 [==============================] - 894s 22s/step - loss: 0.4819 - accuracy: 0.7625
Epoch 8/10
40/40 [==============================] - 893s 22s/step - loss: 0.5584 - accuracy: 0.7344
Epoch 9/10
40/40 [==============================] - 892s 22s/step - loss: 0.4658 - accuracy: 0.7500
Epoch 10/10
40/40 [==============================] - 893s 22s/step - loss: 0.4140 - accuracy: 0.7969

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(train_dataset)
print(f"Train Loss: {loss}, Train Accuracy: {accuracy}")

40/40 [==============================] - 207s 5s/step - loss: 0.3995 - accuracy: 0.7969
Train Loss: 0.39948439598083496, Train Accuracy: 0.796875
